In [2]:
import numpy as np
import pandas as pd
import utm
import math
import pickle
import os
import sys
sys.path.append('./cmps')
sys.path.append('./src/data_cleaning')
import time
import datetime

import CellSense
import load

# Load Data

In [112]:
load = reload(load)
dataset = 'jiading'
datatype = '4g'
data_file = './data/%s_%s/data_%s.csv' % (dataset, datatype, datatype)
gongcan_file = './data/%s_%s/gongcan_%s.csv' % (dataset, datatype, datatype)
disp_path = './display/%s_%s/' % (dataset, datatype)
data_path = './data/%s_%s/' % (dataset, datatype)
map_file = './data/%s_map/%s_EdgeGeometry.txt' % (dataset, dataset)

In [113]:
db = pd.read_csv(data_file)
t = db
x0, x1, y0, y1 = min(t['Latitude']), max(t['Latitude']), min(t['Longitude']), max(t['Longitude'])
area = ((y0, x0), (y1, x1))
area

((121.20118517, 31.28189311), (121.21823488, 31.29332821))

In [133]:
grid_size = 10
default_std = 5

# Train Test Split

In [134]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(db, db[['Longitude', 'Latitude']], test_size=0.33)

In [135]:
columns = [(t + '_%d' % i) for i in range(1, 8) for t in ['RNCID', 'CellID', 'Dbm']]
print columns
X_train[columns].head(10)

['RNCID_1', 'CellID_1', 'Dbm_1', 'RNCID_2', 'CellID_2', 'Dbm_2', 'RNCID_3', 'CellID_3', 'Dbm_3', 'RNCID_4', 'CellID_4', 'Dbm_4', 'RNCID_5', 'CellID_5', 'Dbm_5', 'RNCID_6', 'CellID_6', 'Dbm_6', 'RNCID_7', 'CellID_7', 'Dbm_7']


,RNCID_1,CellID_1,Dbm_1,RNCID_2,CellID_2,Dbm_2,RNCID_3,CellID_3,Dbm_3,RNCID_4,...,Dbm_4,RNCID_5,CellID_5,Dbm_5,RNCID_6,CellID_6,Dbm_6,RNCID_7,CellID_7,Dbm_7
11681,6271,11138581,-81,2147483647,2147483647,-86,2147483647,2147483647,-91,2147483647,...,-95,2147483647,2147483647,-96,2147483647,2147483647,-107,2147483647,2147483647,-98
17827,6271,13073675,-72,2147483647,2147483647,-75,2147483647,2147483647,-97,2147483647,...,-91,-1,-1,-1,-1,-1,-1,-1,-1,-1
15475,6271,11138581,-80,2147483647,2147483647,-92,2147483647,2147483647,-90,2147483647,...,-95,2147483647,2147483647,-102,-1,-1,-1,-1,-1,-1
14850,6271,11138581,-82,2147483647,2147483647,-88,2147483647,2147483647,-92,2147483647,...,-91,2147483647,2147483647,-106,2147483647,2147483647,-105,2147483647,2147483647,-92
11545,6271,13073675,-89,2147483647,2147483647,-94,2147483647,2147483647,-92,2147483647,...,-110,2147483647,2147483647,-99,2147483647,2147483647,-102,-1,-1,-1
10614,6271,8889110,-79,2147483647,2147483647,-95,2147483647,2147483647,-73,2147483647,...,-75,-1,-1,-1,-1,-1,-1,-1,-1,-1
7040,6154,13284373,-82,2147483647,2147483647,-93,2147483647,2147483647,-99,2147483647,...,-101,2147483647,2147483647,-99,2147483647,2147483647,-98,2147483647,2147483647,-92
5084,6154,13284363,-88,2147483647,2147483647,-99,2147483647,2147483647,-95,2147483647,...,-98,2147483647,2147483647,-104,2147483647,2147483647,-99,2147483647,2147483647,-97
16175,6271,11138581,-82,2147483647,2147483647,-89,2147483647,2147483647,-89,2147483647,...,-100,2147483647,2147483647,-100,-1,-1,-1,-1,-1,-1
6828,6154,13284363,-105,2147483647,2147483647,-115,2147483647,2147483647,-113,2147483647,...,-106,2147483647,2147483647,-106,2147483647,2147483647,-111,2147483647,2147483647,-111


In [136]:
len(X_train)

13093

# Model

In [137]:
CellSense = reload(CellSense)
cs = CellSense.CellSense(area, grid_size, default_std, datatype)

In [138]:
cs.fit(X_train)

In [139]:
score, Y_pred = cs.predict(X_test, is_c=False)

In [140]:
error_cellsense = []
cnt = 0
for pt1, pt2 in zip(Y_test.values.tolist(), Y_pred):
    try:
        error_cellsense.append(CellSense.distance(pt1, pt2))
    except:
        print cnt
    cnt += 1

In [142]:
print np.median(error_cellsense)
print np.mean(error_cellsense)

33.8
162.83975810203134


In [36]:
from __future__ import print_function
import os
import pp
import math
import itertools
from collections import namedtuple

def gen_cmd(dataset, datatype, side, def_std, test_size):
    cmd = 'python sacred_sense.py with dataset={} datatype={} side={} def_std={} test_size={}' \
        .format(dataset, datatype, side, def_std, test_size)
    return cmd


def execute(cmd):
    print(cmd)
    res = os.popen(cmd).read()
    print(res)
    return True

In [39]:
ppserver = ()
job_server = pp.Server(ppservers=ppserver)
print("Starting pp with " + str(job_server.get_ncpus()) + " workers")

Config = namedtuple('Config',
                    ['dataset', 'datatype', 'side', 'def_std', 'test_size'])

dataset = ['jiading', 'siping']
datatype = ['2g', '4g']
side = [10, 15, 20, 25, 30, 40]
def_std = [5]
test_size = [0.33, 0.25]

index = 0
jobs = []
for param in itertools.product(dataset, datatype, side, def_std, test_size):
    config = Config(*param)
    cmd = gen_cmd(*config)
    index += 1
    jobs.append([index, cmd, job_server.submit(execute, (cmd,), )])

for index, cmd, job in jobs:
    print("{}/{}: {} Return: {}".format(index, len(jobs), cmd, job()))

job_server.print_stats()


Starting pp with 8 workers
python sacred_sense.py with dataset=jiading datatype=2g side=10 def_std=5 test_size=0.33

1/48: python sacred_sense.py with dataset=jiading datatype=2g side=10 def_std=5 test_size=0.33 Return: True
python sacred_sense.py with dataset=jiading datatype=2g side=10 def_std=5 test_size=0.25

2/48: python sacred_sense.py with dataset=jiading datatype=2g side=10 def_std=5 test_size=0.25 Return: True
python sacred_sense.py with dataset=jiading datatype=2g side=15 def_std=5 test_size=0.33

3/48: python sacred_sense.py with dataset=jiading datatype=2g side=15 def_std=5 test_size=0.33 Return: True
python sacred_sense.py with dataset=jiading datatype=2g side=15 def_std=5 test_size=0.25

4/48: python sacred_sense.py with dataset=jiading datatype=2g side=15 def_std=5 test_size=0.25 Return: True
python sacred_sense.py with dataset=jiading datatype=2g side=20 def_std=5 test_size=0.33

5/48: python sacred_sense.py with dataset=jiading datatype=2g side=20 def_std=5 test_size=0